In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

# functions

In [ ]:
def TimeDiff(df, frameSize): #finds the time difference
    count = 1
    classLabel = df.loc[1,"ClassLabel"]
    time_diff = [0]
    rows = (df.shape)[0]
    for i in range(1,rows):
        if((df.loc[i+1,"ClassLabel"] == classLabel) and (count < frameSize)):
            diff = datetime.datetime.strptime(df.iloc[i,0].split(' ')[3],"%H:%M:%S") - datetime.datetime.strptime(df.iloc[i-1,0].split(' ')[3],"%H:%M:%S")
            time_diff.append(diff.total_seconds())
            count = count + 1
        else:
            time_diff.append(0)
            classLabel = df.loc[i+1,"ClassLabel"]
            count = 1
    df["SNMP Parameters"] = time_diff
    df.rename(index=str, columns={"SNMP Parameters": "Time Difference"})

In [ ]:
def difference(df,cols): #value of each packet is compared with the earlier packet for columns specified in colsToFindDifference and that difference is used as new value for the column
    classLabel = df.loc[1,"ClassLabel"]
    diffList = []

    for j in cols:
        diffList.append([df.loc[1,j]])

    rows = (df.shape)[0]
    for i in range(2,rows+1):
        if((df.loc[i,"ClassLabel"] == classLabel)):
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]] - df.loc[i-1,cols[j]])
        else:
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]])
            classLabel = df.loc[i,"ClassLabel"]

    # print(df)
    # print(df.columns)
    # del df['ClassLabel']
    # print(df)

    for j in range(len(cols)):
        df[cols[j]] = diffList[j]

In [ ]:
def makeNewDataFrame(df, frameSize): #combine 6 packets together
    i = 0
    listOfRowVals = []
    rows = (df.shape)[0]
    while(i+frameSize-1<rows):
        if(df.iloc[i,-1] == df.iloc[i+frameSize-1,-1]):
            rowVals = []
            for j in range(frameSize):
                rowVals = rowVals + list(df.iloc[i+j,:-1])
            rowVals.append(df.iloc[i,-1])
            i = i+frameSize
            listOfRowVals.append(rowVals)
        else:
            j = 1
            while(df.iloc[i,-1] == df.iloc[i+j,-1]):
                j=j+1
            i = i+j
    new_df = pd.DataFrame(listOfRowVals)
    return new_df

# Preprocessing Real time data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load dataset to dataframe



In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/RealTimeData/realTime_SNMP_Dataset.csv')
#To drop iso values
df = df[1:]

# #Coding Class Types into integers
labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 1, 'LAND_Normal' : 2,
            'TCPSYN_Normal' : 3, 'TCPSYNACK_Normal' : 4, 'ICMP_Flood' : 5,
            'UDP_Flood' : 6, 'LAND_Flood' : 7, 'TCPSYN_Flood' : 8,
             'TCPSYNACK_Flood' : 9}
# labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 0, 'LAND_Normal' : 0,
#             'TCPSYN_Normal' : 0, 'TCPSYNACK_Normal' : 0, 'ICMP_Flood' : 1,
#             'UDP_Flood' : 1, 'LAND_Flood' : 1, 'TCPSYN_Flood' : 1,
#              'TCPSYNACK_Flood' : 1}

limit = len(df['ClassLabel']) + 1
for i in range(1, limit):
    df['ClassLabel'][i] = labelDict[df['ClassLabel'][i]]


df = df.loc[0:350,:]
df = df.reset_index(drop=True, )
df.index += 1

print(df.shape)
# print(df)

(350, 66)


In [ ]:
print(df)

In [ ]:
frameSize = 6 #no of packets to analyze at once to detect an attack
TimeDiff(df, frameSize)

# print(df)
# print(df.shape)

In [ ]:
#these are the colums that remained after dimentionality reduction of training data in preprocessing stage. dataframe will only consists of these columns from here onwards
columns=['SNMP Parameters', 'icmp_InEchosReps', 'icmp_InErrors', 'icmp_InMsgs',
       'icmp_InTimeExcds', 'icmp_InTimestamps', 'icmp_OutEchoReps',
       'icmp_OutErrors', 'icmp_OutTimeExcds', 'icmp_OutTimestamps',
       'ip_FragCreates', 'ip_FragOKs', 'ip_InAddrErrors', 'ip_InDelivers',
       'ip_InReceives', 'ip_OutRequests', 'ip_ReasmOKs', 'ip_ReasmReqds',
       'tcp_ActiveOpens', 'tcp_CurrEstablish', 'tcp_EstabResets', 'tcp_InSegs',
       'tcp_OutRsts', 'tcp_OutSegs', 'tcp_PassiveOpens', 'tcp_RetransSegs',
       'udp_InDatagrams', 'udp_NoPorts', 'udp_OutDatagrams',
       'ClassLabel'
       ]

df = df[columns].copy()
# print(df.shape)

In [ ]:
df = df.astype(int) #casting dataframe to integer

# print(len(list((df.loc[:, df.std() == 0]).columns)))
# print(df.shape)
# print((df.loc[:, df.std() == 0]).columns)

In [ ]:
#value of each packet is compared with the earlier packet for columns specified in colsToFindDifference and that difference is used as new value for the column
colsToFindDifference=['icmp_InEchosReps','icmp_InMsgs','icmp_OutErrors','icmp_OutTimestamps','ip_InDelivers','ip_InReceives','ip_OutRequests','udp_InDatagrams','udp_OutDatagrams']
difference(df,colsToFindDifference)
print(df.shape)

(350, 30)


combine packets to one row of data

In [ ]:
df_ = makeNewDataFrame(df, frameSize)
#print(df_)

# Classify Real time data

load the trained classifier model

In [ ]:
import joblib

xtra_multi_classifier=joblib.load('/content/drive/My Drive/Colab Notebooks/A-Prediction-model-for-flooded-packets-in-SNMP-Networks-master/Pickles/pickle_model_xtra.pkl')
# print(xtra_multi_classifier.classes_)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
df_ = df_.iloc[:,1:]
df_ = df_.reset_index(drop=True)
# df_.head()

In [ ]:
print(df_)

classify

In [ ]:
labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 1, 'LAND_Normal' : 2,
            'TCPSYN_Normal' : 3, 'TCPSYNACK_Normal' : 4, 'ICMP_Flood' : 5,
            'UDP_Flood' : 6, 'LAND_Flood' : 7, 'TCPSYN_Flood' : 8,
             'TCPSYNACK_Flood' : 9}
newLabDict={value : key for (key, value) in labelDict.items()}
# print(newLabDict)

In [ ]:
#classify single data point
from sklearn.preprocessing import StandardScaler

test_data=df_.iloc[:,:-1]
sc_ = StandardScaler()
sc_.fit(test_data)
test_data_std = sc_.transform(test_data)
print(xtra_multi_classifier.predict(test_data_std))
class_label=xtra_multi_classifier.predict(test_data_std)[0]


if newLabDict[class_label].find('Flood') != -1:
  print('ATTACK','\nAttack Type : ',newLabDict[class_label])
else:
  print('NORMAL data flow','\nPacket Type : ',newLabDict[class_label])

[5 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 7 7 7 7 7 7 1 1 7 7 7 7 7 1]
ATTACK 
Attack Type :  ICMP_Flood


In [ ]:
# classify all data

from sklearn.preprocessing import StandardScaler

actual=df_.iloc[:,-1]

test_data=df_.iloc[:,:-1]
sc_ = StandardScaler()
sc_.fit(test_data)
test_data_std = sc_.transform(test_data)
print(xtra_multi_classifier.score(test_data_std,actual))

n=0
n2=0
for i in (list(xtra_multi_classifier.predict(test_data_std))):
  print(n,i,actual[n])
  if i==actual[n]:
    n2+=1
  n+=1
print((n2)*100/397)

0.0
0 5 0
1 3 0
2 3 0
3 3 0
4 3 0
5 3 0
6 3 0
7 3 0
8 3 0
9 3 0
10 3 0
11 3 0
12 1 0
13 1 0
14 1 0
15 1 0
16 1 0
17 1 0
18 1 0
19 1 0
20 1 0
21 1 0
22 1 0
23 1 0
24 1 0
25 1 0
26 1 0
27 1 0
28 1 0
29 1 0
30 1 0
31 1 0
32 1 0
33 1 0
34 1 0
35 1 0
36 1 0
37 1 0
38 1 0
39 1 0
40 1 0
41 1 2
42 1 2
43 7 2
44 7 2
45 7 2
46 7 2
47 7 2
48 7 2
49 1 2
50 1 2
51 7 2
52 7 2
53 7 2
54 7 2
55 7 2
56 1 2
0.0
